In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained('mesolitica/nanot5-base-malaysian-cased')

2023-10-04 23:07:43.860847: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-04 23:07:43.935453: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-04 23:07:44.327290: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-04 23:07:44.327333: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [3]:
from glob import glob

checkpoints = sorted(glob('nanot5-tiny-malaysian-cased/checkpoint-*'))
checkpoints

['nanot5-tiny-malaysian-cased/checkpoint-3000000',
 'nanot5-tiny-malaysian-cased/checkpoint-3010000',
 'nanot5-tiny-malaysian-cased/checkpoint-3020000']

In [4]:
model = T5ForConditionalGeneration.from_pretrained(checkpoints[-1])
model.config.eos_token_id = tokenizer.eos_token_id

In [5]:
_ = model.cuda()

In [6]:
s = """
saya tak suka ayam
""".strip()
input_ids = tokenizer.encode(f'terjemah ke Jawi: {s}{tokenizer.eos_token}', return_tensors = 'pt').cuda()
outputs = model.generate(input_ids, max_length = 100)
print(tokenizer.batch_decode(outputs, skip_special_tokens = True))

/home/husein/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


['ساي تق سوك ايم']


In [16]:
from tqdm import tqdm
import json

jawi_left, jawi_right = [], []
rumi_left, rumi_right = [], []

with open('jawi-rumi-news-full.test') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        input_ids = tokenizer.encode(f'terjemah ke Jawi: {l[0]}{tokenizer.eos_token}', return_tensors = 'pt').cuda()
        outputs = model.generate(input_ids, max_length = 2048)
        jawi_left.append(tokenizer.batch_decode(outputs, skip_special_tokens = True)[0])
        jawi_right.append(l[1])
        
        input_ids = tokenizer.encode(f'terjemah ke Rumi: {l[1]}{tokenizer.eos_token}', return_tensors = 'pt').cuda()
        outputs = model.generate(input_ids, max_length = 2048)
        rumi_left.append(tokenizer.batch_decode(outputs, skip_special_tokens = True)[0])
        rumi_right.append(l[0])
        
        if len(rumi_left) >= 10000:
            break

9999it [13:16, 12.55it/s]


In [17]:
from sacrebleu.metrics import BLEU, CHRF, TER

chrf = CHRF(word_order = 2)

In [18]:
chrf.corpus_score(jawi_left, [jawi_right])

chrF2++ = 98.57

In [19]:
chrf.corpus_score(rumi_left, [rumi_right])

chrF2++ = 97.72